In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import fpmax
import itertools
import copy
import numpy as np
from scipy.stats import stats
import random

import os
import sys

ModuleNotFoundError: No module named 'mlxtend'